In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import zipfile

In [ ]:
# 1. Upload de múltiplos arquivos CSV
uploaded = files.upload()

In [5]:
# 2. Criar uma pasta para salvar os gráficos
pasta_saida = "/content/graficos_emg"
os.makedirs(pasta_saida, exist_ok=True)

In [6]:
# Função do filtro média móvel
def media_movel(sinal, janela):
    return np.convolve(sinal, np.ones(janela)/janela, mode='valid')

In [ ]:
# 4. Para cada arquivo CSV enviado
for nome_arquivo in uploaded.keys():
    try:
        # Ler o CSV
        df = pd.read_csv(nome_arquivo)

        # Assumindo 3 colunas: tempo, emg1, emg2
        tempo = df.iloc[:, 0]
        emg1 = df.iloc[:, 1]
        emg2 = df.iloc[:, 2]

        # Extrair o nome do dado (ex: "dados_22")
        nome_base = os.path.splitext(nome_arquivo)[0].replace(" ", "_").lower()  # Ex: dados_(22) -> dados_(22)
        numero_dado = ''.join(filter(str.isdigit, nome_base))  # extrai apenas número

        # Guardar dados para gráfico combinado
        emg1_filtrados = {}
        emg2_filtrados = {}

        for janela in [5, 10, 20]:
            emg1_filtrado = media_movel(emg1, janela)
            emg2_filtrado = media_movel(emg2, janela)
            tempo_filtrado = tempo[janela - 1:]

            # Salva para gráfico geral depois
            emg1_filtrados[janela] = (tempo_filtrado, emg1_filtrado)
            emg2_filtrados[janela] = (tempo_filtrado, emg2_filtrado)

            # Criar gráfico individual
            plt.figure(figsize=(12, 6))

            plt.subplot(2, 1, 1)
            plt.plot(tempo, emg1, label='EMG1 Original', alpha=0.5)
            plt.plot(tempo_filtrado, emg1_filtrado, label=f'EMG1 Filtrado (janela {janela})', color='blue')
            plt.title(f'EMG1 - Janela {janela}')
            plt.legend()

            plt.subplot(2, 1, 2)
            plt.plot(tempo, emg2, label='EMG2 Original', alpha=0.5)
            plt.plot(tempo_filtrado, emg2_filtrado, label=f'EMG2 Filtrado (janela {janela})', color='green')
            plt.title(f'EMG2 - Janela {janela}')
            plt.legend()

            plt.tight_layout()

            nome_saida = f"janela_{janela}_dados_{numero_dado}.png"
            caminho_grafico = os.path.join(pasta_saida, nome_saida)
            plt.savefig(caminho_grafico, dpi=300)
            plt.close()
            print(f"✅ Gráfico salvo: {caminho_grafico}")

        # Criar gráfico combinado ("geral")
        plt.figure(figsize=(12, 6))

        plt.subplot(2, 1, 1)
        plt.plot(tempo, emg1, label="Original", alpha=0.3, color='black')
        for janela, (tempo_filt, emg1_filt) in emg1_filtrados.items():
            plt.plot(tempo_filt, emg1_filt, label=f"Janela {janela}")
        plt.title("EMG1 - Todas as janelas")
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(tempo, emg2, label="Original", alpha=0.3, color='black')
        for janela, (tempo_filt, emg2_filt) in emg2_filtrados.items():
            plt.plot(tempo_filt, emg2_filt, label=f"Janela {janela}")
        plt.title("EMG2 - Todas as janelas")
        plt.legend()

        plt.tight_layout()

        nome_geral = f"janela_geral_dados_{numero_dado}.png"
        caminho_geral = os.path.join(pasta_saida, nome_geral)
        plt.savefig(caminho_geral, dpi=300)
        plt.close()
        print(f"✅ Gráfico geral salvo: {caminho_geral}")

    except Exception as e:
        print(f"❌ Erro no arquivo {nome_arquivo}: {e}")


In [8]:
# 5. Compactar a pasta de saída
caminho_zip = "/content/graficos_emg.zip"
with zipfile.ZipFile(caminho_zip, 'w') as zipf:
    for root, _, files_in_dir in os.walk(pasta_saida):
        for file in files_in_dir:
            full_path = os.path.join(root, file)
            zipf.write(full_path, arcname=os.path.relpath(full_path, pasta_saida))

# 6. Baixar o arquivo ZIP com todos os gráficos
files.download(caminho_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>